# Suite: Purchases
Using Library Python Selenium to test the web site

In [4]:
# ####### DEBUG ONLY #######
# ####### Load the "autoreload" extension #######
%load_ext autoreload
# ####### always reload modules marked with "%import" #######
%autoreload 2

import os
import sys


sys.path.append("../config")
sys.path.append("../util")
sys.path.append("../tests")

In [5]:
# Importing packages
from config import config
from datetime import datetime, timedelta
import json
import logging
import logging.config
import pathlib
from pprint import pprint
import fx_test
from fx_test import TestEngine
import time

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.keys import Keys

import unittest
from ui_helper import UIHelper

## Variables Declaration

In [6]:
log_config = "../config/logging.conf"

# # Create folder "logs"
# pathlib.Path('../logs').mkdir(exist_ok=True)  
# logging.log_file = datetime.now().strftime('../logs/log_%Y%m%d_%H%M%S.log')
logging.config.fileConfig(log_config, disable_existing_loggers=False)
logger = logging.getLogger(__name__)

suite_name = "suite_purchases"
suite_case = "suite_02"
logger.info("Test Suite: {}".format(suite_name))
logger.info("Suite Case: {}".format(suite_case))

[INFO] Test Suite: suite_purchases
[INFO] Suite Case: suite_02


## Functions Definition

### Main Function Begin
def main(args):

In [8]:
mode = config["web"]["mode"]
base_url = config["web"]["base_url"]
timeout = config["web"]["timeout"]
user_name = config["web"]["user_name"]
password = config["web"]["password"]
company_short_name = config["data"]["company_short_name"]
company_name = config["data"]["company_name"]

if mode == "windows":
    driver_path = "../../../library/drivers/chrome/win/chromedriver.exe"
else:
    driver_path = "../../../library/drivers/chrome/mac/chromedriver"
    
logger.info("Mode: {}".format(mode))
logger.info("Base Url: {}".format(base_url))
logger.info("User Name: {}".format(user_name))

[INFO] Mode: windows
[INFO] Base Url: http://211.25.204.108:30002/
[INFO] User Name: zong.xian.soh@adv-fusionex.com


In [9]:
# Get web page
instance = TestEngine(mode="windows", driver_path=driver_path, timeout=timeout)
instance.get_driver().get(base_url)

TestEngine: WebDriver created!
Mode: windows
Driver Path: ../../../library/drivers/chrome/win/chromedriver.exe
Timeout: 10


<a id='top'></a>
## Table of contents
- <a href='#data_load'>Load Test Data</a>
- <a href='#login'>Login</a>
- <a href='#switch_company'>Switch Company</a>
- <a href='#invoices'>Purchases - Purchase Invoices</a>
- <a href='#payments'>Purchases - Payments</a>
- <a href='#debit_notes'>Purchases - Debit Notes</a>
- <a href='#vendors'>Purchases - Vendors</a>

<a id='login'></a>
### Login
<a href='#top'>top</a>

In [ ]:
##Launch uihelper
ui = UIHelper(instance, user_name=user_name, password=password)
ui.login()

<a id='data_load'></a>
### Load Test Data
<a href='#top'>top</a>

In [ ]:
# Load sample data
data_path =  f"../data/{suite_case}/purchases.json"
data = ui.load_data(data_path)

payments = data["payments"]
debit_notes = data["debit_notes"]
invoices = data["purchase_invoices"]
vendors = data["vendors"]

<a id='switch_company'></a>
### Switch Company
<a href='#top'>top</a>

In [ ]:
ui.switch_company(company_name)

<a id='invoices'></a>
### Purchases - Purchase Invoices
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Purchases", "Purchase Invoices")

In [ ]:
for invoice in invoices:
    print(invoice)
    ui.click_add()
    ui.select_dropdown("Vendor", invoice["vendor"])
    ui.select_dropdown("Currency", invoice["currency"])
    ui.select_date_picker("Bill Date", invoice["bill_date"])
    ui.enter_text((By.NAME,"bill_number"), invoice["bill_number"])
    
    locator = (By.XPATH, '//div[@class="card-body"]//div[contains(@class, "col-sm-12 mb-4") and .//*[text()[contains(., "Items")]]]')
    elements = instance.find_elements(locator)
    # to make Items visible
    instance.scroll_to_element(elements[0])
    for i in range(len(invoice["items"])):
        account_type = invoice["items"][i]["account_type"]
        name = invoice["items"][i]["name"]
        quantity = invoice["items"][i]["quantity"]
        # Select Account Type
        instance.click_element((By.XPATH, f'.//tbody/tr[{i+1}]/td[2]//span[@class="el-input__prefix"]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{account_type}"]]') 
        instance.click_element(locator)
        # Select Name
        instance.click_element((By.XPATH, f'.//tbody/tr[{i+1}]/td[3]//span[@class="el-input__prefix"]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{name}"]]') 
        instance.click_element(locator)
        # Insert Quantity
        ui.enter_text((By.NAME, f"items.{i}.quantity"), str(quantity))
    # Add discount
    instance.click_element((By.XPATH, './/tr[@id="tr-discount"]/td/span//span[@class="el-link--inner"]'))
    instance.enter_text((By.XPATH, './/input[@id="pre-discount"]'), invoice["discount_rate"])
    instance.click_element((By.XPATH, './/button[@id="save-discount"]'))
    instance.scroll_down()
    # Select Category
    ui.select_dropdown("Category", invoice["category"])
    ui.click_save()
    instance.wait(5)
    if invoice["status"] == "received":
        instance.click_element((By.XPATH, f'//div[@class="mt-3"]//*[text()[contains(., "Mark Received")]]'))
    if invoice["status"] == "paid":
        locator = (By.XPATH, '//div[@class="card-body"]//div[contains(@class, "timeline-block") and .//*[text()[contains(., "Make Payment")]]]')
        elements = instance.find_elements(locator)
        # to make Make Payments visible
        instance.scroll_to_element(elements[0])
        instance.click_element((By.ID, "button-payment"), parent=elements[0])
        ui.select_date_picker("Date", invoice["payment_date"])
        instance.click_element((By.XPATH, './/div[@class="row"]//div[contains(@class, "form-group") and .//*[text()[contains(., "Account")]]]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{invoice["account"]}"]]') 
        instance.click_element(locator)
        locator = (By.XPATH, './/div[@class="row"]//div[contains(@class, "form-group") and .//*[text()[contains(., "Payment Method")]]]')
        elements=instance.find_elements(locator)
        instance.scroll_to_element(elements[0])
        instance.click_element((By.XPATH, './/div[@class="row"]//div[contains(@class, "form-group") and .//*[text()[contains(., "Payment Method")]]]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{invoice["payment_method"]}"]]') 
        instance.click_element(locator)
        # click save
        locator = (By.XPATH, './/div[@class="modal-content"]//button[@class="btn button-submit btn-success"]')
        instance.click_element(locator)
    instance.click_element((By.XPATH, '//a[@class="nav-link active"]//*[text()[contains(., "Purchase Invoices")]]'))
    instance.wait(5)
    break

<a id='payments'></a>
### Purchases - Payments
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Purchases", "Payments")

In [ ]:
for payment in payments:
    ui.click_add()
    ui.select_date_picker("Date", payment["date"])
    ui.enter_text((By.NAME, "amount"), str(payment["amount"]) + "0000")
    ui.select_dropdown("Account", payment["account"])
    ui.select_dropdown("Vendor", payment["vendor"])
    instance.scroll_down()
    ui.select_dropdown("Category", payment["category"])
    ui.select_dropdown("Recurring", payment["recurring"])
    ui.select_dropdown("Payment Method", payment["payment_method"])
    
    ui.click_save()
    instance.wait(5)

<a id='debit_notes'></a>
### Purchases - Debit Notes
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Purchases", "Debit Notes")

In [ ]:
for debit_note in debit_notes:
    print(debit_note)
    ui.click_add()
    instance.wait(2)
    ui.select_dropdown("Vendor", debit_note["vendor"])
    ui.select_dropdown("Currency", debit_note["currency"])
    ui.select_date_picker("Debit Note Date", debit_note["debit_at"])
    ui.enter_text((By.ID, "debit_note_number"), debit_note["debit_note_number"])
#     ui.select_dropdown("Bill", debit_note["bill_number"])

    locator = (By.XPATH, '//div[@class="card-body"]//div[contains(@class, "col-sm-12 mb-4") and .//*[text()[contains(., "Items")]]]')
    elements = instance.find_elements(locator)
    # to make Items visible
    instance.scroll_to_element(elements[0])
    for i in range(len(debit_note["items"])):
        name = debit_note["items"][i]["name"]
        quantity = debit_note["items"][i]["quantity"]
        # Select Name
        instance.click_element((By.XPATH, f'.//tbody/tr[{i+1}]/td[2]//span[@class="el-input__prefix"]'))
        locator = (By.XPATH, f'//div[@class="el-select-dropdown el-popper" and not(contains(@style, "display: none"))]//li[.//*[text()="{name}"]]') 
        instance.click_element(locator)
        # Insert Quantity
        ui.enter_text((By.NAME, f"items.{i}.quantity"), str(quantity))
    # Add Discount
    instance.click_element((By.XPATH, './/tr[@id="tr-discount"]/td/span//span[@class="el-link--inner"]'))
    instance.enter_text((By.XPATH, './/input[@id="pre-discount"]'), debit_note["discount_rate"])
    instance.click_element((By.XPATH, './/button[@id="save-discount"]'))
    instance.scroll_down()
    # Select Category
    ui.select_dropdown("Category", debit_note["category"])
    ui.click_save()
    instance.wait(5)
    instance.click_element((By.XPATH, f'//a[@class="nav-link active"]//*[text()[contains(., "Debit Notes")]]'))
    instance.wait(2)

<a id='vendors'></a>
### Purchases - Vendors
<a href='#top'>top</a>

In [ ]:
ui.click_side_menu("Purchases", "Vendors")

In [ ]:
for vendor in vendors:
    locator = (By.XPATH, f'.//tbody/tr[contains(@class, "row") and .//a[contains(text(), "{vendor["name"]}")]]')
    elements = instance.find_elements(locator)
    
    if len(elements) > 0:
        logger.info(f"Found: {vendor['name']}")
        instance.scroll_to_element(elements[0])
        ui.click_edit(elements[0])
    else:
        logger.info(f"Not Found: {vendor['name']}")
        ui.click_add()
        
    ui.enter_text((By.ID, "name"), vendor["name"])
    ui.enter_text((By.ID, "email"), vendor["email"])
    ui.enter_text((By.ID, "tax_number"), vendor["tax"])
    ui.select_dropdown("Currency", vendor["currency"])
    ui.enter_text((By.ID, "phone"), vendor["phone"])
    ui.enter_text((By.ID, "website"), vendor["website"])
    ui.enter_text((By.ID, "address"), vendor["address"])
    ui.enter_text((By.ID, "reference"), vendor["reference"])
    ui.click_save()
    instance.wait(5)
    instance.click_element((By.XPATH, f'//a[@class="nav-link active"]//*[text()[contains(., "Vendors")]]'))
    instance.wait(2)

In [ ]:
instance.close()